<a href="https://colab.research.google.com/github/himansig7/IP-Adapter/blob/main/SkinTypeDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
!pip install Pillow

from numpy import asarray

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd
images = '/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types'

/content


In [4]:
#the dataset only comes with images that have been sorted into folders, but not formally labelled
#this function will help to label each of the images as the dataframe is being created
label_index = {"dry": 0, "normal": 1, "oily": 2}
index_label = {0: "dry", 1: "normal", 2: "oily"}

def create_df(base):
    dd = {"images": [], "labels": []}
    for i in os.listdir(base):
        label = os.path.join(base, i)
        for j in os.listdir(label):
            img = os.path.join(label, j)
            dd["images"] += [img]
            dd["labels"] += [label_index[i]]
    return pd.DataFrame(dd)

train_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/train")
val_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/valid")
test_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/test")


In [5]:
def load_image(file_path):
    with Image.open(file_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        return np.array(img)

def extract_image_data(file_path):
    img_array = load_image(file_path)
    height, width, channels = img_array.shape
    return pd.Series({
        'height': height,
        'width': width,
        'channels': channels,
        'image_array': img_array
    })

# Apply the function to each file path in the dataframe
train_df_data = train_df.iloc[:, 0].apply(extract_image_data)
val_df_data = val_df.iloc[:, 0].apply(extract_image_data)
test_df_data = test_df.iloc[:, 0].apply(extract_image_data)

# Join the results with the original dataframe
train_df = train_df.join(train_df_data)
val_df = val_df.join(val_df_data)
test_df = test_df.join(test_df_data)



In [6]:
test_df.head()

,images,labels,height,width,channels,image_array
0,/content/drive/MyDrive/SkincareProject/Oily-Dr...,0,640,640,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,/content/drive/MyDrive/SkincareProject/Oily-Dr...,0,640,640,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
2,/content/drive/MyDrive/SkincareProject/Oily-Dr...,0,640,640,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,/content/drive/MyDrive/SkincareProject/Oily-Dr...,0,640,640,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
4,/content/drive/MyDrive/SkincareProject/Oily-Dr...,0,640,640,3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


In [7]:
x_train = train_df[['image_array','height','width','channels']]
y_train = train_df[['labels']]
x_test = test_df[['image_array','height','width','channels']]
y_test = test_df[['labels']]


In [8]:

# normalize
def normalize(df):
    normalized_images = []
    for index, row in df.iterrows():
      # Extract the image array from the first column
      image_array = row[0]
      # Ensure the array is in float32 format
      image_array = image_array.astype('float32')
      # Normalize the pixel values to the range [0, 1]
      image_array /= 255.0
      # Append the normalized image to a new list
      normalized_images.append(image_array)
    # Create a new DataFrame with the normalized images
    df[0] = normalized_images
    return df

x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

<ipython-input-8-7427f5ecb30e>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_array = row[0]
<ipython-input-8-7427f5ecb30e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[0] = normalized_images
<ipython-input-8-7427f5ecb30e>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_array = row[0]
<ipython-input-8-7427f5ecb30e>:14: SettingWithCopyWarning: 
A value is 

In [24]:
import numpy as np

# Extract the normalized image arrays and convert them to a NumPy array
x_train = np.stack(x_train_norm['image_array'].values)
x_test = np.stack(x_test_norm['image_array'].values)

# Check the shape to ensure it's correct
print(x_train.shape)  # Should output (num_samples, height, width, channels)
print(x_test.shape)
print(y_train.shape)  # Should output (num_samples, num_classes)
print(y_test.shape)

(2756, 640, 640, 3)
(134, 640, 640, 3)
(2756, 1)
(134, 1)


In [29]:
#defining the variables
height = 640
width = 640
num_classes = 3
channels = 3


In [31]:
#imports
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
#validation data
#we need to figure out the whole validation thing

In [32]:
# create CNN structure
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) # notice no MaxPool after this one
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

# compile and train model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_oh, epochs=10, batch_size=32, validation_data=(x_val, y_val_oh)) #this line has error

# evaluate accuracy
y_pred = model.predict(x_test)
y_pred = tf.argmax(y_pred, axis=1)
# y_test = tf.argmax(y_test_oh, axis=1)
loss, acc = model.evaluate(x_test, y_test_oh)
print("Test accuracy:", acc)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_3 (Flatten)         (None, 576)              

NameError: name 'y_train_oh' is not defined